In [ ]:
import numpy as np
import pandas as pd
from scipy.misc import derivative
from scipy.stats import norm
from scipy.integrate import quad

`Payoff function`

$
h(S_T) = S_T^{\frac{1}{3}} + 1.5log(S_T) + 10.0
$

Q1. `Black-Scholes` model:

$\sigma_{LN}$ is constant and determined by the at-the-money option (i.e. S = K).

Assuming that we are evaluating an exotic European derivative using SPX as underlying, S = 3662.45 on 1-Dec-2020, sigma at K = 3660 (nearest strike price) is 0.197 (as per Part 2).

$
S_T = S_0 e^{(r - \frac{\sigma^2}{2})T + \sigma W_T}
$

Substituting above into Payoff function,

$
h(S_T) = S_0^{1/3} e^{((r- \frac{\sigma^2}{2})T + \sigma W_T) \frac{1}{3}} + 1.5 log(S_0) + 1.5(r - \frac{\sigma^2}{2})T + 1.5 \sigma W_T + 10.0
$

$
\Rightarrow E^*[h(S_T)] = [S_0 e^{rT - \frac{\sigma^2 T}{3}}]^\frac{1}{3} + 1.5log(S_0) + 1.5(r - \frac{\sigma^2}{2}) T + 10.0
$

$
V_0 = e^{-rT} E^*[h(S_T)]
$

In [ ]:
# Define parameters for Black-Scholes
S = 3662.45
T = 0.1232876712328767 ### 45/365
r = 0.0020510755555555554 ### based on the interpolated r from Part 2
sigma = 0.197 ### based on implied volatilites of SPX plotted in Part 2; at K=3660

BlackScholesExoticPrice(S, r, T, sigma)  #### to double check

37.703496294931355

In [ ]:
def BlackScholesExoticPrice(S, r, T, sigma):
    expPayoff = (S * np.exp(T*(r - sigma**2/3))) ** (1/3) + 1.5*np.log(S) + 1.5*(r-sigma**2/2)*T + 10.0
    price = np.exp(-r*T)*expPayoff 
    return price

BlackScholesExoticPrice(S, r, T, sigma)

37.703496294931355

In [ ]:
# test for Black-Scholes using static replication
S = 3662.45
T = 0.1232876712328767 ### 45/365
r = 0.0020510755555555554 ### based on the interpolated r from Part 2
sigma = 0.197 ### based on implied volatilites of SPX plotted in Part 2; at K=3660
F = S * np.exp(r*T)

def BlackScholesCall(S, K, r, sigma, T):
    d1 = (np.log(S/K)+(r+sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    return S*norm.cdf(d1) - K*np.exp(-r*T)*norm.cdf(d2)

def BlackScholesPut(S, K, r, sigma, T):
    return BlackScholesCall(S, K, r, sigma, T) - S + K*np.exp(-r*T)

def callintegrand(K, S, r, T, sigma):
    price = BlackScholesCall(S, K, r, sigma, T) * ((-2/9)*(K**(-5/3)) - (1.5)*(1/K**2)) 
    return price

def putintegrand(K, S, r, T, sigma):
    price = BlackScholesPut(S, K, r, sigma, T) * ((-2/9)*(K**(-5/3)) - (1.5)*(1/K**2)) 
    return price

I_put = quad(lambda x: putintegrand(x, S, r, T, sigma), 0.0, F)
I_call = quad(lambda x: callintegrand(x, S, r, T, sigma), F, 5000) 

BlackScholesExoticPriceStatic = np.exp(-r*T)*((F**(1/3)) + 1.5*np.log(F) + 10) + I_put[0] + I_call[0] 
BlackScholesExoticPriceStatic

37.70349629793738

Q2. `Bachelier Model`: 



$
S_T = S_0 + \sigma W_T
$

Substituting above into Payoff function,

$
h(S_T) = (S_0 +\sigma W_T)^{1/3} + 1.5 log(S_0 + \sigma W_T) + 10.0
$


In [ ]:
def BachelierIntegrand(x, sigma, T, S_0):
    S_T = S_0 + sigma*(T**0.5)*x
    expPayoff = (S_T)**(1/3) + 1.5*np.log(S_T) + 10.0
    f_x = np.exp(-x**2 / 2) / ((2*np.pi)**0.5)
    return expPayoff * f_x

I = quad(lambda x: BachelierIntegrand(x, 0.197, 0.1232876712328767, 3662.45), 0, 5000)
BachelierExoticPrice = np.exp(-r*T) * 2*I[0]

BachelierExoticPrice

37.7136968236079

$$\int_{-\infty}^{\infty} x^\frac{1}{3}\frac{1}{\sigma\sqrt{2T\pi}}e^{-\frac{1}{2}(\frac{x-S_0}{\sigma\sqrt{T}})^2} \; dx$$

In [ ]:
def integrand1(x, sigma, T, S):
    g_x = np.log(x)
    f_x = 1 / (sigma * (2 * T * np.pi) ** 0.5) * np.exp(-0.5 * ((x - S) / (sigma * T ** 0.5)) ** 2)
    return g_x * f_x

I = quad(lambda x: integrand1(x, 0.197, 0.1232876712328767, 3662.45), 0, 5000)
print('The integral is: %.9f' % I[0])

In [ ]:
# Define parameters for Bachelier
S = 3662.45
T = 0.1232876712328767 ### 45/365
r = 0.0020510755555555554 ### based on the interpolated r from Part 2
sigma = 0.197 ### based on implied volatilites of SPX plotted in Part 2; at K=3660
F = S * np.exp(r*T)

def BachelierCall(S, K, r, sigma, T):
    temp = (S - K)/ (sigma *np.sqrt(T))
    return np.exp(-r*(T)) * ((S-K)*norm.cdf(temp) + sigma*np.sqrt(T)*norm.pdf(temp))

def BachelierPut(S, K, r, sigma,T):
    temp = (-S + K)/ (sigma *np.sqrt(T))
    return np.exp(-r*(T)) * ((K - S)*norm.cdf(temp) + sigma*np.sqrt(T)*norm.pdf(temp))

def callintegrand(K, S, r, T, sigma):
    price = BachelierCall(S, K, r, sigma, T) * ((-2/9)*(K**(-5/3)) - (1.5)*(1/K**2)) 
    return price

def putintegrand(K, S, r, T, sigma):
    price = BachelierPut(S, K, r, sigma, T) * ((-2/9)*(K**(-5/3)) - (1.5)*(1/K**2)) 
    return price

I_put = quad(lambda x: putintegrand(x, S, r, T, sigma), 0.0, F)
I_call = quad(lambda x: callintegrand(x, S, r, T, sigma), F, 5000) 

BachelierExoticPrice = np.exp(-r*T)*((F**(1/3)) + 1.5*np.log(F) + 10) + I_put[0] + I_call[0] 
BachelierExoticPrice

37.71527504035332

Q3. Static replication of European payoff using `SABR Model` (calibrated in Part 2 of project): 

To value the exotic European derivative, we just need to substitute the $\sigma_{SABR}$ into the Black-Scholes formula. 

`Payoff function`

$
h(S_T) = S_T^{\frac{1}{3}} + 1.5log(S_T) + 10.0
$

$
\Rightarrow h(F) = F^{\frac{1}{3}} + 1.5 * log(F) + 10.0
$
, where $F = S_0 e^{rT}$

$
h'(S_T) = \frac{1}{3} S_T^{-\frac{2}{3}} + 1.5\frac{1}{S_T}
$

$
h''(S_T) = -\frac{2}{9} S_T^{-\frac{5}{3}} - 1.5S_T^{-2}
$

$
\Rightarrow h''(K) = -\frac{2}{9} K^{-\frac{5}{3}} - 1.5K^{-2}
$


For any twice-differentiable payoff $h(S_T)$, Breeden-Litzenberger states that

$
V_0 = e^{-rT}h(F) + \int_0^{F}h''(K)P(K)\;dK + \int_{F}^{\infty}h''(K)C(K)\;dK
$

$
V_0 = e^{-rT}(F^{\frac{1}{3}} + 1.5 * \log(F) + 10.0) + \int_0^{F}(-\frac{2}{9}K^{-\frac{5}{3}} - 1.5K^{-2})P(K)\;dK + \int_F^\infty(-\frac{2}{9}K^{-\frac{5}{3}} - 1.5K^{-2})C(K)\;dK
$

$V_0 = e^{-rT}((S_0e^{rT})^{\frac{1}{3}} + 1.5 * \log(S_0e^{rT}) + 10.0) + \int_0^{(S_0e^{rT})}(-\frac{2}{9}K^{-\frac{5}{3}} - 1.5K^{-2})P(K)\;dK + \int_F^\infty(-\frac{2}{9}K^{-\frac{5}{3}} - 1.5K^{-2})C(K)\;dK$

In [ ]:
#SABR function

def SABR(F, K, T, alpha, beta, rho, nu):
    X = K
    # if K is at-the-money-forward
    if abs(F - K) < 1e-12:
        numer1 = (((1 - beta)**2)/24)*alpha*alpha/(F**(2 - 2*beta))
        numer2 = 0.25*rho*beta*nu*alpha/(F**(1 - beta))
        numer3 = ((2 - 3*rho*rho)/24)*nu*nu
        VolAtm = alpha*(1 + (numer1 + numer2 + numer3)*T)/(F**(1-beta))
        sabrsigma = VolAtm
    else:
        z = (nu/alpha)*((F*X)**(0.5*(1-beta)))*np.log(F/X)
        zhi = np.log((((1 - 2*rho*z + z*z)**0.5) + z - rho)/(1 - rho))
        numer1 = (((1 - beta)**2)/24)*((alpha*alpha)/((F*X)**(1 - beta)))
        numer2 = 0.25*rho*beta*nu*alpha/((F*X)**((1 - beta)/2))
        numer3 = ((2 - 3*rho*rho)/24)*nu*nu
        numer = alpha*(1 + (numer1 + numer2 + numer3)*T)*z
        denom1 = ((1 - beta)**2/24)*(np.log(F/X))**2
        denom2 = (((1 - beta)**4)/1920)*((np.log(F/X))**4)
        denom = ((F*X)**((1 - beta)/2))*(1 + denom1 + denom2)*zhi
        sabrsigma = numer/denom

    return sabrsigma

In [ ]:
def SABRCall(S, K, r, alpha, beta, rho, nu, T):
    sabr_vol = SABR(S*np.exp(r*T), K, T, alpha, beta, rho, nu)
    return BlackScholesCall(S, K, r, sabr_vol, T)


def SABRPut(S, K, r, alpha, beta, rho, nu, T):
    sabr_vol = SABR(S*np.exp(r*T), K, T, alpha, beta, rho, nu)
    return BlackScholesPut(S, K, r, sabr_vol, T)


def sabrcallintegrand(K, S, r, T, alpha, beta, rho, nu):
    price = SABRCall(S, K, r, alpha, beta, rho, nu, T) / K**2
    return price


def sabrputintegrand(K, S, r, T, alpha, beta, rho, nu):
    price = SABRPut(S, K, r, alpha, beta, rho, nu, T) / K**2
    return price

# Calibrated SABR model parameters based on Part 2
alpha = 1.817 
beta = 0.7 
rho = -0.404 
nu = 2.790

# Define remaining parameters
S = 3662.45
T = 0.1232876712328767 ### 45/365
r = 0.0020510755555555554 ### based on the interpolated r from Part 2
F = S * np.exp(r*T)


I_put = quad(lambda x: sabrputintegrand(x, S, r, T, alpha, beta, rho, nu), 1e-6, F)
I_call = quad(lambda x: sabrcallintegrand(x, S, r, T, alpha, beta, rho, nu), F, 5000)
sabrExoticPrice = np.exp(-r*T)*((F**(1/3)) + 1.5*np.log(F) + 10) + I_put[0] + I_call[0] 
sabrExoticPrice


37.718442900958145

### Other workings below

In [ ]:
def BlackScholesPrice(S, K, r, sigma, T):
    # e^(-rT)h(F)
    F = S * np.exp(r * T)
    term1 = np.exp(-r * T) * (F ** (1/3) + 1.5 * np.log(F) + 10.0)

    # 1st integral
    integrand1 = ((-2/9) * (x ** (-5 /3)) - 1.5 * x ** (-2)) * BlackScholesPut(S, K, r, sigma, T)
    # figure out how to integrate haha
    term2 = integrate_put(integrand1)

    # 2nd integral
    integrand2 = ((-2/9) * (x ** (-5 /3)) - 1.5 * x ** (-2)) * BlackScholesCall(S, K, r, sigma, T)
    term3 = integrate_call(integrand2)

    return term1 + term2 + term3

"Model-free" integrated variance 

$
\sigma_{MF}^2 T = E[\int_{0}^{T} \sigma_t^2 dt]
= \int_{0}^{T} \sigma^2 dt
= \sigma^2 T
$

$   
\mathbb{E}\left[\int_0^T\sigma_t^2 \;dt\right] = 2e^{rT} \left(\int_0^{F}\frac{P(K)}{K^2}\;dK + \int_{F}^{\infty}\frac{C(K)}{K^2}\;dK\right)
$

In [ ]:
# to determine sigma 

def BlackScholesCall(S, K, r, sigma, T):
    d1 = (np.log(S/K)+(r+sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    return S*norm.cdf(d1) - K*np.exp(-r*T)*norm.cdf(d2)


def BlackScholesPut(S, K, r, sigma, T):
    return BlackScholesCall(S, K, r, sigma, T) - S + K*np.exp(-r*T)

def callintegrand(K, S, r, T, sigma):
    price = BlackScholesCall(S, K, r, sigma, T) / K**2 
    return price


def putintegrand(K, S, r, T, sigma):
    price = BlackScholesPut(S, K, r, sigma, T) / K**2
    return price

In [ ]:
# to test with parameters 

S = 100
r = 0.04
T = 1.0
sigma = 0.5
F = S * np.exp(r*T)

I_put = quad(lambda x: putintegrand(x, S, r, T, sigma), 0.0, F)
I_call = quad(lambda x: callintegrand(x, S, r, T, sigma), F, 5000) 
E_var = 2*np.exp(r*T)*(I_put[0] + I_call[0])
print('The expected integrated variance is: %.9f' % E_var)


The expected integrated variance is: 0.250000000


In [ ]:
# 1: Black-Scholes Model
# to determine price of derivative


S = 3662.45
T = 0.1232876712328767 ### 45/365
r = 0.0020510755555555554 ### based on the interpolated r from Part 2
sigma = 0.197 ### based on implied volatilites of SPX plotted in Part 2; at K=3660
F = S * np.exp(r*T)

def BlackScholesCall(S, K, r, sigma, T):
    d1 = (np.log(S/K)+(r+sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    return S*norm.cdf(d1) - K*np.exp(-r*T)*norm.cdf(d2)


def BlackScholesPut(S, K, r, sigma, T):
    return BlackScholesCall(S, K, r, sigma, T) - S + K*np.exp(-r*T)

def BScallintegrand(K, S, r, T, sigma):
    price = BlackScholesCall(S, K, r, sigma, T) * ((-2/9) * K**(-5/3) - 1.5*(1/K**2)) 
    return price

def BSputintegrand(K, S, r, T, sigma):
    price = BlackScholesPut(S, K, r, sigma, T) * ((-2/9) * K**(-5/3) - 1.5*(1/K**2))
    return price

I_put = quad(lambda x: BSputintegrand(x, S, r, T, sigma), 0.0, F)
I_call = quad(lambda x: BScallintegrand(x, S, r, T, sigma), F, 5000)
E_var = 2*np.exp(r*T)*(I_put[0] + I_call[0])
print('The expected integrated variance is: %.9f' % E_var)

The expected integrated variance is: -0.023563443


In [ ]:
I_put = quad(lambda x: BSputintegrand(x, S, r, T, sigma), 0.0, F)
I_call = quad(lambda x: BScallintegrand(x, S, r, T, sigma), F, 5000) 

V_0 = np.exp(-r*T)*((F**(-1/3)) + 1.5*np.log(F) + 10) + I_put[0] + I_call[0] 
V_0

In [ ]:

S = 100.0
r = 0.05
T = 1.0
sigma = 0.4
F = S * np.exp(r*T)
I_put = quad(lambda x: putintegrand(x, S, r, T, sigma), 0.0, F)
I_call = quad(lambda x: callintegrand(x, S, r, T, sigma), F, 5000)
E_var = 2*np.exp(r*T)*(I_put[0] + I_call[0])
print('The expected integrated variance is: %.9f' % E_var)

Under the Black-Scholes Model, 

$
S_T = S_0 e^{(r-\frac{\sigma^2}{2})T + \sigma W_T}
$

Valuation formula: 

$V_0 = e^{-rT} E[payoff]$

substituting $S_T$ into payoff function, 

$V_0 = e^{-rT} E[S_0^{1/3}e^{(r-\frac{\sigma^2}{2})\frac{T}{3} + \frac{\sigma W_T}{3}} + 1.5logS_0 + 1.5(r - \frac{\sigma^2}{2})T +1.5\sigma W_T +10]$

$V_0 = e^{-rT} \left[ S_0^{1/3}e^{\frac{rT}{3}} + 1.5logS_0 + 1.5(r - \frac{\sigma^2}{2})T + 10 \right]$

$V_0 = S_0^{1/3} e^{\frac{-2rT}{3}} + 1.5 e^{-rT} logS_0 + 1.5 e^{-rT} (r - \frac{\sigma^2}{2})T + 10e^{-rT} $

Under the Bachelier Model, 

$
S_T = S_0 + \sigma W_T
$

Valuation formula: 

$V_0 = e^{-rT} E[payoff]$

substituting $S_T$ into payoff function,

$V_0 = e^{-rT} E[(S_0 + \sigma W_T)^{1/3} + 1.5 log(S_0 + \sigma W_T) + 10]$



For any twice-differentiable payoff $h(S_T)$, Breeden-Litzenberger states that

$
V_0 = e^{-rT}h(F) + \int_0^{F}h''(K)P(K)\;dK + \int_{F}^{\infty}h''(K)C(K)\;dK
$
  
To implement Breeden-Litzenberger formula (or static replication in general) in Python, we need to be able to handle numerical integration for the put/call integrals.

We need to use numerical integration to evaluate the formula. This can be done in Python easily. For example, consider the integral

$
\int_0^1 x^2 \; dx = \frac{1}{3}
$

We can evaluate it in Python as follows:

In [ ]:
from scipy.integrate import quad


def integrand(x):
    return x**2


I = quad(integrand, 0.0, 1.0)
print('The integral is: %.9f' % I[0])


The integral is: 0.333333333


We can test our static replication implementation with

$
\mathbb{E}\left[\int_0^T\sigma_t^2 \;dt\right] = 2e^{rT} \left(\int_0^{F}\frac{P(K)}{K^2}\;dK + \int_{F}^{\infty}\frac{C(K)}{K^2}\;dK\right)
$

Suppose we assume that market is following Black76 model, then we have

In [ ]:
import numpy as np
from scipy.stats import norm


def BlackScholesCall(S, K, r, sigma, T):
    d1 = (np.log(S/K)+(r+sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    return S*norm.cdf(d1) - K*np.exp(-r*T)*norm.cdf(d2)


def BlackScholesPut(S, K, r, sigma, T):
    return BlackScholesCall(S, K, r, sigma, T) - S + K*np.exp(-r*T)


def callintegrand(K, S, r, T, sigma):
    price = BlackScholesCall(S, K, r, sigma, T) / K**2
    return price


def putintegrand(K, S, r, T, sigma):
    price = BlackScholesPut(S, K, r, sigma, T) / K**2
    return price


S = 100.0
r = 0.05
T = 1.0
sigma = 0.4
F = S * np.exp(r*T)
I_put = quad(lambda x: putintegrand(x, S, r, T, sigma), 0.0, F)
I_call = quad(lambda x: callintegrand(x, S, r, T, sigma), F, 5000)
E_var = 2*np.exp(r*T)*(I_put[0] + I_call[0])
print('The expected integrated variance is: %.9f' % E_var)


The expected integrated variance is: 0.160000000


Since we are using Black76 model in the above, we can cross-check our implementation with the integrated variance based on Black76 model, since under Black76 (or Black-Scholes) model, volatility is deterministic.
  
Hence we have
\begin{equation*}
  \begin{split}
    \mathbb{E}\left[ \int_0^T \sigma_t^2 \; dt \right] &= \int_0^T \sigma^2 \; dt \\
    &= \sigma^2T
  \end{split}
\end{equation*}

So for instance if $\sigma=0.4$ and $T=1$, then the integrated variance should be $\sigma^2T = 0.16$.

Once tested, we should replace BlackScholes with SABR model in order to accurately capture the implied volatility smiile/skew in the option market.

In [ ]:
V = np.exp(rT)*E(payoff)

NameError: name 'np' is not defined

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=84dba4f4-47dd-4672-95f1-e2f4f6eed8bc' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>